In [3]:
import pandas as pd
import os
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline as ImbPipeline

In [4]:
train_data = pd.read_csv('/Users/marlenawasiak/Desktop/Data_Collection/UNSW_NB15_training-set.csv')
test_data = pd.read_csv('/Users/marlenawasiak/Desktop/Data_Collection/UNSW_NB15_testing-set.csv')

In [ ]:
X_train = train_data.drop(columns=['attack_cat'])
y_train = train_data['attack_cat']
X_test = test_data.drop(columns=['attack_cat'])
y_test = test_data['attack_cat']
categorical_features = ['proto', 'service', 'state']
X_train = pd.get_dummies(X_train, columns=categorical_features, drop_first=True)
X_test = pd.get_dummies(X_test, columns=categorical_features, drop_first=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)
oversampler = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)
scaler = StandardScaler()
X_train_resampled = scaler.fit_transform(X_train_resampled) 
X_test_scaled = scaler.transform(X_test)                     


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_resampled), y=y_train_resampled)
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}
model = Sequential()
model.add(Dense(128, input_dim=X_train_resampled.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train_resampled, y_train_resampled, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test), class_weight=class_weight_dict)


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 11s 592us/step - accuracy: 0.6430 - loss: 0.8994 - val_accuracy: 0.6394 - val_loss: 1.3868
Epoch 2/50
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 11s 622us/step - accuracy: 0.7240 - loss: 0.6669 - val_accuracy: 0.6317 - val_loss: 2.1254
Epoch 3/50
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 11s 635us/step - accuracy: 0.7337 - loss: 0.6376 - val_accuracy: 0.7726 - val_loss: 1.6026
Epoch 4/50
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 11s 601us/step - accuracy: 0.7381 - loss: 0.6215 - val_accuracy: 0.7698 - val_loss: 1.8644
Epoch 5/50
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 11s 652us/step - accuracy: 0.7421 - loss: 0.6072 - val_accuracy: 0.7093 - val_loss: 2.5264
Epoch 6/50
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 11s 644us/step - accuracy: 0.7455 - loss: 0.5999 - val_accuracy: 0.7764 - val_loss: 2.3490
Epoch 7/50
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 11s 639us/step - accuracy: 0.7471 - loss: 0.5928 - val_accuracy: 0.7909 - val_loss: 2.7585
Epoch 8/50
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 11s 654us/s

In [ ]:

y_pred_encoded = model.predict(X_test_scaled).argmax(axis=1)
test_accuracy = accuracy_score(y_test, y_pred_encoded)
print("Test Accuracy:", test_accuracy)
class_report = classification_report(
    y_test, 
    y_pred_encoded, 
    labels=range(len(label_encoder.classes_)), 
    target_names=label_encoder.classes_, 
    zero_division=1
)
print("Classification Report - Clean Data:\n", class_report)


2573/2573 ━━━━━━━━━━━━━━━━━━━━ 1s 301us/step
Test Accuracy: 0.6788976339697809
Classification Report - Clean Data:
                 precision    recall  f1-score   support

      Analysis       0.06      0.21      0.09       677
      Backdoor       0.05      0.55      0.09       583
           DoS       0.25      0.11      0.15      4089
      Exploits       0.35      0.50      0.41     11132
       Fuzzers       0.49      0.64      0.55      6062
       Generic       1.00      0.33      0.50     18871
        Normal       1.00      1.00      1.00     37000
Reconnaissance       0.58      0.62      0.60      3496
     Shellcode       0.20      0.43      0.27       378
         Worms       0.12      0.20      0.15        44

      accuracy                           0.68     82332
     macro avg       0.41      0.46      0.38     82332
  weighted avg       0.80      0.68      0.70     82332



In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

X_train = train_data.drop(columns=['attack_cat'])
y_train = train_data['attack_cat']
X_test = test_data.drop(columns=['attack_cat'])
y_test = test_data['attack_cat']
categorical_features = ['proto', 'service', 'state']
X_train = pd.get_dummies(X_train, columns=categorical_features, drop_first=True)
X_test = pd.get_dummies(X_test, columns=categorical_features, drop_first=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)
print("Before augmentation:", X_train.shape, y_train.shape)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
print("After SMOTE augmentation:", X_train_resampled.shape, y_train_resampled.shape)
noise = np.random.normal(0, 0.01, X_train_resampled.shape)
X_train_noisy = X_train_resampled + noise
X_train_augmented = np.vstack((X_train_resampled, X_train_noisy))
y_train_augmented = np.hstack((y_train_resampled, y_train_resampled))

print("After adding noise:", X_train_augmented.shape, y_train_augmented.shape)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_augmented)  
X_test_scaled = scaler.transform(X_test)                 
class_weights = compute_class_weight(
    'balanced', classes=np.unique(y_train_augmented), y=y_train_augmented
)
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}
model = Sequential()
model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(
    X_train_scaled, y_train_augmented, 
    epochs=50, 
    batch_size=32, 
    validation_data=(X_test_scaled, y_test), 
    class_weight=class_weight_dict
)
print("\nModel evaluation on test set:")
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


Before augmentation: (175341, 193) (175341,)
After SMOTE augmentation: (560000, 193) (560000,)
After adding noise: (1120000, 193) (1120000,)


/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
35000/35000 ━━━━━━━━━━━━━━━━━━━━ 18s 498us/step - accuracy: 0.6897 - loss: 0.7856 - val_accuracy: 0.8215 - val_loss: 0.6884
Epoch 2/50
35000/35000 ━━━━━━━━━━━━━━━━━━━━ 18s 510us/step - accuracy: 0.7607 - loss: 0.6037 - val_accuracy: 0.7370 - val_loss: 1.6781
Epoch 3/50
35000/35000 ━━━━━━━━━━━━━━━━━━━━ 17s 487us/step - accuracy: 0.7744 - loss: 0.5721 - val_accuracy: 0.7420 - val_loss: 1.8948
Epoch 4/50
35000/35000 ━━━━━━━━━━━━━━━━━━━━ 17s 485us/step - accuracy: 0.7805 - loss: 0.5571 - val_accuracy: 0.7446 - val_loss: 2.3958
Epoch 5/50
35000/35000 ━━━━━━━━━━━━━━━━━━━━ 16s 464us/step - accuracy: 0.7859 - loss: 0.5445 - val_accuracy: 0.6139 - val_loss: 6.5876
Epoch 6/50
35000/35000 ━━━━━━━━━━━━━━━━━━━━ 17s 486us/step - accuracy: 0.7889 - loss: 0.5361 - val_accuracy: 0.6023 - val_loss: 6.7252
Epoch 7/50
35000/35000 ━━━━━━━━━━━━━━━━━━━━ 16s 456us/step - accuracy: 0.7930 - loss: 0.5270 - val_accuracy: 0.6820 - val_loss: 3.9844
Epoch 8/50
35000/35000 ━━━━━━━━━━━━━━━━━━━━ 18s 504us/s

In [ ]:
import numpy as np
import tensorflow as tf
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent, CarliniL2Method
from art.estimators.classification import TensorFlowV2Classifier
from sklearn.metrics import accuracy_score, classification_report

critical_features = [40, 174, 159, 31, 25]
mask = np.zeros(X_train_resampled.shape[1], dtype=np.float32)
mask[critical_features] = 1.0
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=len(label_encoder.classes_),
    input_shape=(X_train_resampled.shape[1],),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy()
)
print("\n--- FGSM Attack ---")
fgsm = FastGradientMethod(estimator=classifier, eps=3.0, targeted=False)
X_test_adv_fgsm = fgsm.generate(X_test_scaled, mask=mask)
fgsm_preds = model.predict(X_test_adv_fgsm).argmax(axis=1)
fgsm_accuracy = accuracy_score(y_test, fgsm_preds)
fgsm_class_report = classification_report(y_test, fgsm_preds, target_names=label_encoder.classes_, zero_division=1)
print(f"FGSM Adversarial Accuracy: {fgsm_accuracy}")
print(f"Classification Report on FGSM Adversarial Examples:\n{fgsm_class_report}")




--- FGSM Attack ---
2573/2573 ━━━━━━━━━━━━━━━━━━━━ 1s 264us/step
FGSM Adversarial Accuracy: 0.526356702132828
Classification Report on FGSM Adversarial Examples:
                precision    recall  f1-score   support

      Analysis       0.04      0.03      0.03       677
      Backdoor       0.01      0.05      0.01       583
           DoS       0.05      0.10      0.06      4089
      Exploits       0.39      0.43      0.41     11132
       Fuzzers       0.12      0.08      0.09      6062
       Generic       0.96      0.03      0.05     18871
        Normal       0.78      1.00      0.88     37000
Reconnaissance       0.10      0.04      0.06      3496
     Shellcode       0.01      0.10      0.02       378
         Worms       0.00      0.00      0.00        44

      accuracy                           0.53     82332
     macro avg       0.25      0.18      0.16     82332
  weighted avg       0.64      0.53      0.47     82332



In [ ]:

print("\n--- PGD Attack ---")
pgd = ProjectedGradientDescent(estimator=classifier, eps=2.0, eps_step=0.1, max_iter=100, targeted=False)
X_test_adv_pgd = pgd.generate(X_test_scaled, mask=mask)
pgd_preds = model.predict(X_test_adv_pgd).argmax(axis=1)
pgd_accuracy = accuracy_score(y_test, pgd_preds)
pgd_class_report = classification_report(y_test, pgd_preds, target_names=label_encoder.classes_, zero_division=1)
print(f"PGD Adversarial Accuracy: {pgd_accuracy}")
print(f"Classification Report on PGD Adversarial Examples:\n{pgd_class_report}")



--- PGD Attack ---


PGD - Batches: 0it [00:00, ?it/s]

2573/2573 ━━━━━━━━━━━━━━━━━━━━ 1s 262us/step
PGD Adversarial Accuracy: 0.543276004469708
Classification Report on PGD Adversarial Examples:
                precision    recall  f1-score   support

      Analysis       0.05      0.07      0.06       677
      Backdoor       0.03      0.27      0.06       583
           DoS       0.06      0.15      0.09      4089
      Exploits       0.52      0.33      0.40     11132
       Fuzzers       0.11      0.12      0.11      6062
       Generic       1.00      0.13      0.23     18871
        Normal       0.83      1.00      0.91     37000
Reconnaissance       0.04      0.04      0.04      3496
     Shellcode       0.01      0.10      0.02       378
         Worms       0.02      0.16      0.03        44

      accuracy                           0.54     82332
     macro avg       0.27      0.24      0.20     82332
  weighted avg       0.69      0.54      0.53     82332



In [ ]:
print("\n--- Carlini & Wagner Attack (on a subset) ---")
subset_indices = []
classes_to_include = np.unique(y_test)
for cls in classes_to_include:
    indices = np.where(y_test == cls)[0][:20]  
    subset_indices.extend(indices)

subset_indices = np.array(subset_indices)
X_test_subset = X_test_scaled[subset_indices]
y_test_subset = y_test[subset_indices]
cw = CarliniL2Method(classifier=classifier, confidence=1.0, targeted=False, max_iter=100)
X_test_adv_cw_subset = cw.generate(X_test_subset, mask=mask)
cw_preds_subset = model.predict(X_test_adv_cw_subset).argmax(axis=1)
cw_accuracy_subset = accuracy_score(y_test_subset, cw_preds_subset)
cw_class_report_subset = classification_report(
    y_test_subset, 
    cw_preds_subset, 
    target_names=label_encoder.classes_, 
    zero_division=1
)

print(f"Carlini & Wagner Adversarial Accuracy (Subset): {cw_accuracy_subset}")
print(f"\nClassification Report on Carlini & Wagner Adversarial Examples (Subset):\n{cw_class_report_subset}")
results = {
    "clean": {
        "accuracy": clean_accuracy,
        "report": clean_class_report
    },
    "fgsm": {
        "accuracy": fgsm_accuracy,
        "report": fgsm_class_report
    },
    "pgd": {
        "accuracy": pgd_accuracy,
        "report": pgd_class_report
}
}
results["cw_subset"] = {
    "accuracy": cw_accuracy_subset,
    "report": cw_class_report_subset
}

np.save("dynamic_feature_mask.npy", mask)



--- Carlini & Wagner Attack (on a subset) ---


C&W L_2:   0%|          | 0/200 [00:00<?, ?it/s]

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 504us/step
Carlini & Wagner Adversarial Accuracy (Subset): 0.285

Classification Report on Carlini & Wagner Adversarial Examples (Subset):
                precision    recall  f1-score   support

      Analysis       1.00      0.00      0.00        20
      Backdoor       0.27      0.60      0.37        20
           DoS       0.67      0.10      0.17        20
      Exploits       0.19      0.35      0.25        20
       Fuzzers       0.23      0.40      0.29        20
       Generic       1.00      0.30      0.46        20
        Normal       0.29      1.00      0.45        20
Reconnaissance       0.00      0.00      0.00        20
     Shellcode       1.00      0.00      0.00        20
         Worms       1.00      0.10      0.18        20

      accuracy                           0.28       200
     macro avg       0.56      0.29      0.22       200
  weighted avg       0.56      0.28      0.22       200



NameError: name 'clean_accuracy' is not defined

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler, LabelEncoder, QuantileTransformer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import numpy as np
import pandas as pd

X_train = train_data.drop(columns=['attack_cat'])
y_train = train_data['attack_cat']
X_test = test_data.drop(columns=['attack_cat'])
y_test = test_data['attack_cat']
categorical_features = ['proto', 'service', 'state']
X_train = pd.get_dummies(X_train, columns=categorical_features, drop_first=True)
X_test = pd.get_dummies(X_test, columns=categorical_features, drop_first=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train_encoded)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=42)
X_train_scaled = quantile_transformer.fit_transform(X_train_scaled)
X_test_scaled = quantile_transformer.transform(X_test_scaled)
num_classes = len(np.unique(y_train_encoded))
mlp_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')  
])

mlp_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = mlp_model.fit(X_train_scaled, y_train_resampled, epochs=30, batch_size=32, validation_data=(X_test_scaled, y_test_encoded))
y_pred_encoded = mlp_model.predict(X_test_scaled).argmax(axis=1)
y_pred = label_encoder.inverse_transform(y_pred_encoded)
y_test_decoded = label_encoder.inverse_transform(y_test_encoded)

print("\nConfusion Matrix:")
print(confusion_matrix(y_test_decoded, y_pred))

print("\nClassification Report:")
print(classification_report(y_test_decoded, y_pred))
critical_features = [40, 174, 159, 31, 25] 
mask = np.zeros(X_train_scaled.shape[1], dtype=np.float32)
mask[critical_features] = 1.0
classifier = TensorFlowV2Classifier(
    model=mlp_model,
    nb_classes=num_classes,
    input_shape=(X_train_scaled.shape[1],),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy()
)

print("\n--- FGSM Attack ---")
fgsm = FastGradientMethod(estimator=classifier, eps=3.0, targeted=False)
X_test_adv_fgsm = fgsm.generate(X_test_scaled, mask=mask)
fgsm_preds = mlp_model.predict(X_test_adv_fgsm).argmax(axis=1)
fgsm_accuracy = accuracy_score(y_test_encoded, fgsm_preds)
fgsm_class_report = classification_report(y_test_encoded, fgsm_preds, target_names=label_encoder.classes_, zero_division=1)
print(f"FGSM Adversarial Accuracy: {fgsm_accuracy}")
print(f"\nClassification Report on FGSM Adversarial Examples:\n{fgsm_class_report}")




Epoch 1/30


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


17500/17500 ━━━━━━━━━━━━━━━━━━━━ 9s 469us/step - accuracy: 0.6416 - loss: 0.9254 - val_accuracy: 0.8374 - val_loss: 0.4727
Epoch 2/30
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 8s 461us/step - accuracy: 0.7164 - loss: 0.7025 - val_accuracy: 0.8351 - val_loss: 0.4942
Epoch 3/30
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 8s 456us/step - accuracy: 0.7265 - loss: 0.6741 - val_accuracy: 0.8360 - val_loss: 0.4865
Epoch 4/30
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 8s 454us/step - accuracy: 0.7317 - loss: 0.6607 - val_accuracy: 0.8332 - val_loss: 0.5217
Epoch 5/30
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 8s 462us/step - accuracy: 0.7366 - loss: 0.6476 - val_accuracy: 0.8326 - val_loss: 0.5561
Epoch 6/30
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 8s 457us/step - accuracy: 0.7407 - loss: 0.6399 - val_accuracy: 0.7717 - val_loss: 0.7752
Epoch 7/30
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 8s 458us/step - accuracy: 0.7454 - loss: 0.6323 - val_accuracy: 0.8428 - val_loss: 0.8465
Epoch 8/30
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 8s 460us/step - accuracy: 0.7

In [ ]:

print("\n--- PGD Attack ---")
pgd = ProjectedGradientDescent(estimator=classifier, eps=3.0, eps_step=0.1, max_iter=100, targeted=False)
X_test_adv_pgd = pgd.generate(X_test_scaled, mask=mask)
pgd_preds = mlp_model.predict(X_test_adv_pgd).argmax(axis=1)
pgd_accuracy = accuracy_score(y_test_encoded, pgd_preds)
pgd_class_report = classification_report(y_test_encoded, pgd_preds, target_names=label_encoder.classes_, zero_division=1)
print(f"PGD Adversarial Accuracy: {pgd_accuracy}")
print(f"\nClassification Report on PGD Adversarial Examples:\n{pgd_class_report}")
print("\n--- Carlini & Wagner Attack (on a subset) ---")



--- PGD Attack ---


PGD - Batches: 0it [00:00, ?it/s]

2024-11-23 20:50:24.212260: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


2573/2573 ━━━━━━━━━━━━━━━━━━━━ 1s 272us/step
PGD Adversarial Accuracy: 0.6777923529125978

Classification Report on PGD Adversarial Examples:
                precision    recall  f1-score   support

      Analysis       0.06      0.26      0.10       677
      Backdoor       0.02      0.18      0.04       583
           DoS       0.10      0.18      0.13      4089
      Exploits       0.44      0.17      0.25     11132
       Fuzzers       0.36      0.23      0.28      6062
       Generic       0.98      0.73      0.83     18871
        Normal       0.92      1.00      0.96     37000
Reconnaissance       0.41      0.18      0.25      3496
     Shellcode       0.05      0.24      0.08       378
         Worms       0.01      0.16      0.01        44

      accuracy                           0.68     82332
     macro avg       0.34      0.33      0.29     82332
  weighted avg       0.75      0.68      0.69     82332


--- Carlini & Wagner Attack (on a subset) ---


In [ ]:


subset_indices = []
classes_to_include = np.unique(y_test_encoded)
for cls in classes_to_include:
    indices = np.where(y_test_encoded == cls)[0][:20]  
    subset_indices.extend(indices)
subset_indices = np.array(subset_indices)
X_test_subset = X_test_scaled[subset_indices]
y_test_subset = y_test_encoded[subset_indices]
cw = CarliniL2Method(classifier=classifier, confidence=1.0, targeted=False, max_iter=100)
X_test_adv_cw_subset = cw.generate(X_test_subset, mask=mask)
cw_preds_subset = mlp_model.predict(X_test_adv_cw_subset).argmax(axis=1)
cw_accuracy_subset = accuracy_score(y_test_subset, cw_preds_subset)
cw_class_report_subset = classification_report(
    y_test_subset, 
    cw_preds_subset, 
    target_names=label_encoder.classes_, 
    zero_division=1
)

print(f"Carlini & Wagner Adversarial Accuracy (Subset): {cw_accuracy_subset}")
print(f"\nClassification Report on Carlini & Wagner Adversarial Examples (Subset):\n{cw_class_report_subset}")

C&W L_2:   0%|          | 0/200 [00:00<?, ?it/s]

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 641us/step
Carlini & Wagner Adversarial Accuracy (Subset): 0.51

Classification Report on Carlini & Wagner Adversarial Examples (Subset):
                precision    recall  f1-score   support

      Analysis       0.00      0.00      0.00        20
      Backdoor       0.25      0.85      0.38        20
           DoS       0.25      0.05      0.08        20
      Exploits       0.36      0.45      0.40        20
       Fuzzers       0.59      0.50      0.54        20
       Generic       1.00      0.75      0.86        20
        Normal       0.65      1.00      0.78        20
Reconnaissance       0.65      0.55      0.59        20
     Shellcode       1.00      0.25      0.40        20
         Worms       0.93      0.70      0.80        20

      accuracy                           0.51       200
     macro avg       0.57      0.51      0.48       200
  weighted avg       0.57      0.51      0.48       200

